In [104]:
import tkinter as tk
import tkinter.font
import numpy as np
import random
import time
import openpyxl
from datetime import datetime

#요일 설정
days = ["월", "화", "수", "목", "금", "토", "일"]

#불러오기_학생 이름
wb = openpyxl.load_workbook("studentsNames.xlsx")
ws = wb.active
names = []
for row in range(2, ws.max_row+1):
    names.append((ws.cell(row = row, column = 1).value))
scores = []
for row in range(2, ws.max_row+1):
    scores.append((ws.cell(row = row, column = 2).value))
dict_students = dict(zip(names, scores))

#불러오기_쿠폰
coupons = []
file_coupon = open("Coupon.txt", "r", encoding = "UTF-8")
for coupon in file_coupon.readlines():
    coupons.append(coupon.strip())
    
def reset():
    for name in list(dict_students.keys()):
        dict_students[name] = 0
    for index, btn in enumerate(button):
    # Create the button
        btn.config(text= names[index] + str(dict_students[names[index]]),
                   font=font_name)
        
#함수_제비추첨
def drawing_lots():
    list_students = []
    for name, count in dict_students.items():
        list_students.append([name] * count)
    list_students = sum(list_students, [])
    global lucky_student
    lucky_student= list_students[random.randint(0, len(list_students)-1)]
    result = ("😁", lucky_student, coupons[random.randint(0, len(coupons)-1)], "!")
    lbl_result.config(text=result)

#함수_1씩 증가
def click_button(index):
    # Increment the value in the list
    dict_students[names[index]] += 1
    # Update the text on the label
    button[index].config(text=list(dict_students.keys())[index] + str(dict_students[names[index]]))
    
def temporary_save():
    for index, value in enumerate(list(dict_students.items())):
        ws.cell(row = index+2, column = 2, value = value[1])
    wb.save("studentsNames.xlsx")
    window.destroy()
    
def final_save():
    file_excel = openpyxl.load_workbook("history.xlsx")
    sheet = file_excel.active
    date = datetime.now().strftime("%Y.%m.%d")
    day = days[datetime.now().weekday()]
    for name, num in dict_students.items():
        if name == lucky_student:
            sheet.append([date, day, name, num, 1])
        else:
            sheet.append([date, day, name, num, 0])
    file_excel.save("history.xlsx")
    window.destroy()

# Create the main window
window = tk.Tk()
font_name=tkinter.font.Font(family="맑은 고딕", size=12)
font_click=tkinter.font.Font(family="맑은 고딕", size=15, slant="italic", weight="bold")
font_result=tkinter.font.Font(family="맑은 고딕", size=15, weight = "bold")
window.title("발표 횟수를 높이면 당첨 확률이 올라간다!")
window.geometry("400x750+0+0")
window.resizable(True, True)


# Create the buttons and labels using a for loop
button = []
label = []
for i in range(len(dict_students)):
    # Create the button
    button.append(tk.Button(window, text=names[i] + str(dict_students[names[i]]),
                            font=font_name,
                            command=lambda i=i: click_button(i)))
    button[i].grid(row=i // 3, column=i % 3, ipadx = 20, ipady=3)
    
tk.Button(window, text="뽑기", command=drawing_lots,
         font=font_click,
         bg="black",
         fg="white").grid(row = (len(names) // 3)+1, column = 1)

tk.Button(window, text="리셋", command=reset,
         font=font_click,
         bg="black",
         fg="white").grid(row = (len(names) // 3)+2, column = 1)

tk.Button(window, text="임시저장 후 종료", command=temporary_save,
         font=font_click,
         bg="black",
         fg="white").grid(row = (len(names) // 3)+3, column = 1)

tk.Button(window, text="저장 후 종료", command=final_save,
         font=font_click,
         bg="black",
         fg="white").grid(row = (len(names) // 3)+4, column = 1)

lbl_result = tk.Label(window, text="",
                      bd=5,
                      font=font_result)
lbl_result.grid(row = (len(names) // 3)+5,
                column = 1,
                rowspan=3)

window.mainloop()